In [11]:
import os
from pathlib import Path
import pandas as pd
import re
import nibabel as nib

from mri_data import utils
from statistics import mean

In [6]:
ensemble_dir = Path("/mnt/h/srs-9/aschoplex/test1/work_dir/working_directory_prediction_finetuning/ensemble_prediction")
predictions = [file.name for file in os.scandir(ensemble_dir)]
prediction_subs = []
for file in predictions:
    sub = re.search(r"MRI_(\d{4}).+", file)[1]
    prediction_subs.append(int(sub))

prediction_labels = [ensemble_dir / file for file in predictions]

In [5]:
subjects_file = "/home/srs-9/Projects/ms_mri/analysis/paper1/data0/manual_labels.csv"
dataroot = Path("/mnt/h/3Tpioneer_bids")
df = pd.read_csv(subjects_file, index_col="subid")

gt_labels = []
for sub in prediction_subs:
    sub_root = dataroot / df.loc[sub, "sub-ses"]
    gt_label = sub_root / "choroid_t1_flair-CH.nii.gz"
    if not gt_label.is_file():
        gt_label = sub_root / "choroid_t1_flair-ED.nii.gz"
    gt_labels.append(gt_label)

In [9]:
dice_scores = []
for sub, gt, pred in zip(prediction_subs, gt_labels, prediction_labels):
    gt_im = nib.load(gt).get_fdata()
    pred_im = nib.load(pred).get_fdata()
    dice_scores.append((sub, utils.dice_score(gt_im, pred_im)))

In [10]:
for entry in dice_scores:
    print("{}: {:0.2f}".format(entry[0], entry[1]))

1011: 0.75
1029: 0.70
1033: 0.69
1037: 0.65
1080: 0.71
1085: 0.75
1087: 0.82
1089: 0.77
1109: 0.71
1119: 0.61
1163: 0.71
1191: 0.72
1234: 0.66
1265: 0.68
1272: 0.58
1280: 0.73
1293: 0.71
1321: 0.73
1437: 0.62
1518: 0.71
1540: 0.68
1547: 0.70
1548: 0.70
2081: 0.73
2083: 0.65
2126: 0.72
2132: 0.74
2144: 0.68
2164: 0.73
2187: 0.74


In [12]:
dices = [entry[1] for entry in dice_scores]
print(mean(dices))

0.7030011397084672


In [14]:
swinunet_result_file = "/home/srs-9/Projects/ms_mri/analysis/choroid_pineal_pituitary_crosstrain_t1/dataframe.csv"
df = pd.read_csv(swinunet_result_file, index_col="subid")

swin_unet_dices = df.loc[prediction_subs, "choroid_dice"]

df_compare = pd.DataFrame({"swinunet": swin_unet_dices, "aschoplex": dices}, index=prediction_subs)

In [16]:
df['choroid_dice'].mean()

0.698428828299513